# 2022-09-23 analysis

This study examines the dependence of the x-y-z hollowing on the MEBT optics (QV01, QH02, QV03, QH04). I ran various cases and saved the initial/final bunch and rms history.

In [ ]:
import sys
import os
from tqdm.notebook import tqdm
from tqdm.notebook import trange
import importlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import proplot as pplt
from ipywidgets import widgets
from ipywidgets import interactive

sys.path.append('/Users/46h/Research/')
from btfsim.analysis.utils import load_bunch
from btfsim.analysis.utils import load_history
from beamphys import plotting as mplt
from beamphys import utils
from beamphys import image as bi
from beamphys import dist as bd

## Settings

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['savefig.dpi'] = 300
# pplt.rc['pdf.fonttype'] = 42

In [ ]:
folder = '/Users/46h/Dropbox (ORNL)/work/btf/btf-sim/2022-09-23_RFQbunch_MEBT1_vary_optics/'    
os.listdir(folder)

In [ ]:
fig_path = os.path.join(folder, 'figures')
if not os.path.isdir(fig_path):
    os.mkdir(fig_path)

In [ ]:
def save(figname, t=None):
    if t is not None:
        figname = t + '_' + figname
    plt.savefig(os.path.join(fig_path, f'{figname}.png'))
    return

## Load data

In [ ]:
timestamps = [
    '220923123019',  # nominal optics,
    '220923151759',  # flipped quad polarities
    '220923183321',  # quadrupoles turned off
    '220924194223',  # x-x' <--> y-y' in inital beam
    '220926112450',  # x' --> -x' in initial beam
    '220926122126',  # y' --> -y' in initial beam
]

def get_prefix(timestamp):
    return f'{timestamp}-sim-0-HZ04'

timestamp = '220926122126'
prefix = get_prefix(timestamp)
node = 'HZ04'

In [ ]:
filenames = {
    'bunch': {},
    'history': os.path.join(folder, f'{prefix}-history.dat'),
}
nodes = ['init', 'HZ04']
for node in nodes:
    filenames['bunch'][node] = os.path.join(folder, f'{prefix}-bunch-{node}.dat')
    
filenames

In [ ]:
history = load_history(filenames['history'])
history.head()

## History

In [ ]:
fig_kws = dict(figsize=(2.75, 2))
plot_kws = dict(marker='.', ms=1, lw=0)

In [ ]:
fig, ax = pplt.subplots(**fig_kws)
for dim in ['x', 'y', 'z']:
    data = history[f'eps_{dim}'].values
    data /= data[0]
    ax.plot(history['s'], data, label=r'$\varepsilon_{}$'.format(dim), **plot_kws)
ax.legend(ncol=1, loc='upper left', ms=3)
ax.format(xlabel='Distance [m]', ylabel='Relative growth')
save(f'{timestamp}_relative_rms_emittance_growth')

In [ ]:
fig, ax = pplt.subplots(**fig_kws)
for i, dim in enumerate(['x', 'y', 'z']):
    i *= 2
    col = f'sig_{i}{i}'
    data = np.sqrt(history[col].values)
    data *= 1e3  # convert [m] to [mm]
    ax.plot(history['s'], data, label=r'${}$'.format(dim), **plot_kws)
ax.legend(ncol=1, loc='upper left')
ax.format(xlabel='Distance [m]', ylabel='RMS beam size [mm]')
save(f'{timestamp}_rms_beam_size')

In [ ]:
fig, axes = pplt.subplots(ncols=2, figsize=(5, 2), sharey=False)
for ax, param in zip(axes, ['beta', 'alpha']):
    for dim in ['x', 'y']:
        col = f'{param}_{dim}'
        xdata = history['s'].values
        ydata = history[col].values
        ax.plot(xdata, ydata, label=r'$\{}_{}$'.format(param, dim))
    ax.legend(ncol=1, loc='upper left', handlelength=1.5)
axes.format(xlabel='Distance [m]')
axes[0].format(ylabel='[m/rad]')
save(f'{timestamp}_twiss_beta_alpha')

### Compare rms beam size and emittance growth in three different experiments 

In [ ]:
_timestamps = [
    '220926122126',  # x diverging, y diverging
    '220923123019',  # x diverging, y converging
    '220924194223',  # x converging, y diverging
    '220926112450',  # x converging, y converging
]
histories = [load_history(os.path.join(folder, f'{get_prefix(t)}-history.dat'))
             for t in _timestamps]

plot_kws = dict(
    marker='.', lw=0, ms=1,
)

fig, axes = pplt.subplots(ncols=len(histories), nrows=2, figwidth=6.5, spany=False, aligny=True)
for j, history in enumerate(histories):
    for i, dim in enumerate(['x', 'y', 'z']):
        eps = history[f'eps_{dim}'].values
        rms = 1e3 * np.sqrt(history[f'sig_{2 * i}{2 * i}'].values)
        axes[1, j].plot(history['s'], eps / eps[0], label=r'${}$'.format(dim), **plot_kws)
        axes[0, j].plot(history['s'], rms / rms[0], label=r'${}$'.format(dim), **plot_kws)
axes[1, 0].legend(ncol=1, loc='upper left', ms=3)
axes[0, 0].format(ylabel='Rel. beam size')
axes[1, 0].format(ylabel='Rel. emittance')
axes.format(
    xlabel='Distance [m]',
    toplabels=['x+, y+', 'x+, y-', 'x-, y+', 'x-, y-'],
)
save('conv_div_compare_rms')
plt.show()

## Bunch

In [ ]:
dims = ["x", "x'", "y", "y'", "z", "dE"]
units = ["mm", "mrad", "mm", "mrad", "mm", "keV"]
labels = [f'{d} [{u}]' for d, u in zip(dims, units)]

In [ ]:
node = 'HZ04'
X = load_bunch(filenames['bunch'][node])

### Interactive

In [ ]:
prof_kws=dict(lw=0.7, alpha=0.6, color='white', scale=0.09)

mplt.interactive_proj2d_discrete(
    X,
    nbins=40,
    dims=dims,
    units=units,
    prof_kws=prof_kws,
)

### x-y-z 

In [ ]:
limits = mplt.auto_limits(X, zero_center=True)
limits

In [ ]:
from beamphys import dist

bins = [40, 40, 40]

for axis in [(0, 2, 4), (0, 4, 2), (2, 4, 0)]:

    H, edges = bd.histogram(X[:, axis], bins=bins, binrange=[limits[i] for i in axis])
    centers = [utils.get_bin_centers(e) for e in edges]
    H = H / np.max(H)

    ncols = 5
    trim = int(0.33 * (H.shape[-1]))
    ii = np.linspace(trim, H.shape[-1] - 1 - trim, ncols).astype(int)

    fig, axes = pplt.subplots(ncols=ncols, figwidth=(8.0 / 5.0)*ncols)
    for ax, i in zip(axes, ii):
        mplt.plot_image(H[:, :, i], x=centers[0], y=centers[1], ax=ax, vmax=1.0)
        text = rf'{dims[axis[-1]]}$\approx {centers[-1][i]:.1f}$ [{units[axis[-1]]}]'
        # ax.annotate(text, xy=(0.02, 0.98), xycoords='axes fraction', color='white', verticalalignment='top')
        ax.format(title=text, title_kw=dict(fontsize='medium'))
    axes.format(
        xlabel=labels[axis[0]],
        ylabel=labels[axis[1]],
    )
    save(f'{timestamp}_{node}_{dims[axis[0]]}_{dims[axis[1]]}_{dims[axis[-1]]}slice.png')

### Corner

In [ ]:
cmap = pplt.Colormap('mono', left=0.065, right=0.9)
cmap

In [ ]:
axes = mplt.corner(
    X, labels=labels, kind='hist',
    mask_zero=True,
    cmap=cmap,
)
axes.format(xlabel_kw=dict(fontsize='large'), ylabel_kw=dict(fontsize='large'))
save(f'{timestamp}_corner_{node}')